# Food Classification with CNN - Building a Restaurant Recommendation System

This assignment focuses on developing a deep learning-based food classification system using Convolutional Neural Networks (CNNs). You will build a model that can recognize different food categories and use it to return the food preferences of a user.

## Learning Objectives
- Implement CNNs for image classification
- Work with real-world food image datasets
- Build a preference-detector system

## Background: AI-Powered Food Preference Discovery

The system's core idea is simple:

1. Users upload 10 photos of dishes they enjoy
2. Your CNN classifies these images into the 91 categories
3. Based on these categories, the system returns the user's taste profile

Your task is to develop the core computer vision component that will power this detection engine.

You are given a training ("train" folder) and a test ("test" folder) dataset which have ~45k and ~22k samples respectively. For each one of the 91 classes there is a subdirectory containing the images of the respective class.

## Assignment Requirements

### Technical Requirements
- Implement your own pytorch CNN architecture for food image classification
- Use only the provided training dataset split for training
- Train the network from scratch ; No pretrained weights can be used
- Report test-accuracy after every epoch
- Report all hyperparameters of final model
- Use a fixed seed and do not use any CUDA-features that break reproducibility
- Use Pytorch 2.6

### Deliverables
1. Jupyter Notebook with CNN implementation, training code etc.
2. README file
3. Report (max 3 pages)

Submit your report, README and all code files as a single zip file named GROUP_[number]_NC2425_PA. The names and IDs of the group components must be mentioned in the README.
Do not include the dataset in your submission.

### Grading

1. Correct CNN implementation, training runs on the uni computers (computer rooms DM.0.07, DM.0.13, DM.0.17, DM.0.21) according to the README.MD instructions without ANY exceptions: 3pt
2. Perfect 1:1 reproducibility on those machines: 1pt
3. Very clear github-repo-style README.MD with instructions for running the code: 1pt
4. Report: 1pt
5. Model test performance on test-set: interpolated from 30-80% test-accuracy: 0-3pt
6. Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt
7. Bonus point: use an LLM (API) to generate short description / profile of preferences of the simulated user

**The main reason why we mention the machines in the uni computer rooms is to make it clear to you how we will test your submissions and to make you aware that you can use these machines for working on your assignment if you do not have access to a computer with enough gpu. You do not have to use these machines for this assignment. Using the specified pytorch version, not enabling torch.backends.cuda.matmul.allow_tf32, setting fixed seeds and making sure your code doesn't use more than 8gb VRAM should allow you to fulfill the reproducibility criteria.**

**(If there is anything unclear about this assignment please post your question in the Brightspace discussions forum or send an email)**


In [52]:
import pandas
import torch
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageOps
import os
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision.transforms import ToTensor
from torchvision import datasets
from torch.utils.data import TensorDataset, DataLoader, Dataset


In [53]:
#os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"   # deterministic mode for GPU

random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)


# Loading the datasets
The dataset is already split into a train and test set in the directories "train" and "test".

In [54]:
# different dataloader approach to allow for data augmentation

class FoodDataset(Dataset):
    def __init__(self, root_dir, transform=None, encoder=None):
        self.images = []
        self.labels = []
        self.transform = transform
        self.encoder = encoder

        string_labels = []
        for folder in os.scandir(root_dir): # iterate through the contents of the directory
            if not folder.is_dir(): # make sure content is directory (could be .DS_Store file)
                continue
            for image in os.scandir(folder):
                string_labels.append(folder.name) # an image's parent folder name is its label
                self.images.append(image)
        self.labels = self.encoder.transform(string_labels)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.open(self.images[idx])
        image = ImageOps.grayscale(image)
        label = self.labels[idx]
        if self.transform: # make transformations
            image = self.transform(image)
        return image, label

    def get_normalization_values(self):
        """
        calculate the mean and standard deviation of the image data to normalize it
        :return: mean and standard deviation
        """
        pixel_sum = 0.0 # for rgb: np.zeros(3)
        pixel_squared_sum = 0.0 # for rgb: np.zeros(3)
        pixel_count = 0
        for image in self.images:
            img = Image.open(image).resize((64, 64))
            img = ImageOps.grayscale(img)  # Either remove or use convert RGB
            img = np.array(img, dtype=np.float32) / 255.0  # shape: (H, W)
            pixel_sum += img.sum()
            pixel_squared_sum += (img ** 2).sum() # add axis (0,1) or something
            pixel_count += img.size # shape[0] * shape[1]

        mean = pixel_sum / pixel_count
        std = np.sqrt(pixel_squared_sum / pixel_count - mean ** 2)
        return mean, std

    def set_transform(self, transform):
        """
        sets transformation pipeline
        :param transform: transformation pipeline
        :return:
        """
        self.transform = transform

# get list of all class names
class_names = []
for folder in os.scandir("train"):
    if folder.is_dir():
        class_names.append(folder.name)

# encode class strings into integer tensors
encoder = LabelEncoder()
encoder.fit(class_names)
labels = encoder.fit_transform(class_names)

print("making train_dataset")
train_dataset = FoodDataset(root_dir="train", encoder=encoder)
print("making test_dataset\n")
test_dataset  = FoodDataset(root_dir="test", encoder=encoder)

print("calculating train normalization values")
train_mean, train_std = train_dataset.get_normalization_values()
print(f"mean = {train_mean}, std = {train_std}\n")

# transformations pipeline for train dataset
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64, 64)), # resize
    torchvision.transforms.RandomHorizontalFlip(), # chance to randomly flip image
    torchvision.transforms.RandomRotation(15), # chance to randomly rotate
    torchvision.transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2), # randomly apply color changes
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(train_mean, train_std) # normalize according to mean and std
])

print("calculating test normalization values:")
test_mean, test_std = test_dataset.get_normalization_values()
print(f"mean = {train_mean}, std = {train_std}\n")

# transformation pipeline for test dataset (no random augmentations here)
test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64, 64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(test_mean, test_std)
])

print("setting transforms")
train_dataset.set_transform(train_transforms)
test_dataset.set_transform(test_transforms)

print("making dataloaders")
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=16, shuffle=False)
print("done")

# test dataset
x, y = next(iter(train_loader))
print("Input shape:", x.shape)  # Expect: torch.Size([16, 1, 224, 224])
print("Input range:", x.min().item(), "to", x.max().item())  # Expect: ~-2 to 2
print("Labels:", y[:10])  # Expect: Tensor of class indices between 0 and 90
print("Label dtype:", y.dtype)  # Expect: torch.int64

making train_dataset
making test_dataset

calculating train normalization values
mean = 0.46283867955207825, std = 0.2549550235271454

calculating test normalization values:
mean = 0.46283867955207825, std = 0.2549550235271454

setting transforms
making dataloaders
done
Input shape: torch.Size([16, 1, 64, 64])
Input range: -1.8153737783432007 to 1.9992165565490723
Labels: tensor([42, 50, 88, 81, 81, 56, 61, 17,  2, 48])
Label dtype: torch.int64


In [55]:
# small scale dataset for test
# only run if you want it (not recommended)
from torch.utils.data import Subset

subset = Subset(train_dataset, list(range(32)))
tiny_loader = DataLoader(subset, batch_size=8, shuffle=True)

# CNN Implementation

In [58]:
# original cnn
def get_num_classes(f):
    class_count = 0
    for folder in os.scandir(f): # iterate through the contents of the directory
        if not folder.is_dir(): # make sure content is directory (could be .DS_Store file)
            continue
        class_count += 1
    return class_count
batch_size = 16 # 16: safe, 32: faster but might not run on every pc
num_classes = get_num_classes("train")
learning_rate = 0.001
num_epochs = 20

class FoodCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),  # first argument 1 for gray 3 for rgb
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 32, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                 # 3×224×224 → 32×112×112

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),                 # 64×56×56
        )

        # compute flattened size using dummy tensor with only 0s
        dummy = torch.zeros(1, 1, 64, 64) # second argument 1 for gray 3 for rgb
        flat = self.features(dummy).numel()

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(flat, 128),
            nn.ReLU(inplace=True),
            nn.Dropout(0.3), # randomly ignores part of the neurons during train (not eval) in order to prevent overfitting -- can be removed or changed
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Training the model
Implement your training process below. Report the test-accuracy after every epoch for the training run of the final model.

Hint: before training your model make sure to reset the seed in the training cell, as otherwise the seed may have changed due to previous training runs in the notebook

Note: If you implement automatic hyperparameter tuning, split the train set into train and validation subsets for the objective function.

In [59]:
# check if GPU is available and use it if possible, run on CPU otherwise

import torch
print(torch.__version__, torch.version.cuda)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))


2.7.0+cu128 12.8
True
NVIDIA GeForce RTX 4070 Laptop GPU


In [60]:
     # reset the seed in this cell
def accuracy(logits, labels):
    preds = logits.argmax(dim=1)
    return (preds == labels).float().mean().item() * 100

def evaluate(model, loader):
    model.eval() # sets model into eval mode (disables dropout and weight change etc)
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for x, y in loader: # for each batch
            x, y = x.to(device), y.to(device)
            acc_sum += accuracy(model(x), y) * x.size(0)
            n += x.size(0)

            # show first five predictions for each batch
            # preds = model(x).argmax(dim=1)
            # print("Sample predictions:", preds[:5].tolist())
            # print("As labels:", encoder.inverse_transform(preds[:5].tolist()))

    return acc_sum / n


device = "cuda" if torch.cuda.is_available() else "cpu" # use gpu if available
model = FoodCNN(num_classes).to(device)

criterion  = nn.CrossEntropyLoss()
# optimizer  = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=0.005, momentum = 0.9) # optimizer from practical classes
#scheduler  = optim.lr_scheduler.StepLR(optimizer, step_size=8, gamma=0.1)

best = 0.0
for epoch in range(1, num_epochs):
    model.train() # set model into training mode (enable weight change, dropout etc)
    print("training epoch:", epoch)
    running_loss = 0.0
    for x, y in train_loader: # for each batch

        x, y = x.to(device), y.to(device)

        # training steps i think
        loss = criterion(model(x), y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    test_acc = evaluate(model, test_loader)
    print(f"epoch {epoch:02d} | test acc {test_acc:.2f}%")
    print(f"Epoch {epoch:02d} | Train loss: {running_loss:.4f}")

    if test_acc > best:
        torch.save(model.state_dict(), "best_model.pth")
        best = test_acc

    #scheduler.step()


training epoch: 1
epoch 01 | test acc 2.60%
Epoch 01 | Train loss: 12822.3368
training epoch: 2
epoch 02 | test acc 3.39%
Epoch 02 | Train loss: 12593.6609
training epoch: 3
epoch 03 | test acc 4.11%
Epoch 03 | Train loss: 12466.8840
training epoch: 4
epoch 04 | test acc 4.08%
Epoch 04 | Train loss: 12367.4563
training epoch: 5
epoch 05 | test acc 5.03%
Epoch 05 | Train loss: 12292.2072
training epoch: 6
epoch 06 | test acc 6.03%
Epoch 06 | Train loss: 12189.0708
training epoch: 7
epoch 07 | test acc 6.63%
Epoch 07 | Train loss: 12069.4848
training epoch: 8
epoch 08 | test acc 6.92%
Epoch 08 | Train loss: 11984.1359
training epoch: 9


KeyboardInterrupt: 

# Calculating model performance
Load the best version of your model ( which should be produced and saved by previous cells ), calculate and report the test accuracy.

In [61]:
# Load the best model weights
model = FoodCNN(num_classes).to(device)
model.load_state_dict(torch.load("best_model.pth"))

final_test_acc = evaluate(model, test_loader)
print(f"Final Test Accuracy: {final_test_acc:.2f}%")


Final Test Accuracy: 6.92%


# Summary of hyperparameters
Report the hyperparameters ( learning rate etc ) that you used in your final model for reproducibility.

# Simulation of random user
Pick 10 random pictures of the test set to simulate a user uploading images and report which categories occur how often in these: 1pt

In [62]:
def img_to_tensor(img, mean, std):
    """
    makes image tensor with necessary transformations
    :param img: path to image
    :param mean: mean from image parent folder (probably test_mean)
    :param std: standard deviation from folder (probably test_std)
    :return:
    """
    # transformation pipeline
    transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((64, 64)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean, std)
    ])

    # open and apply all transformations
    image = Image.open(img)
    image = ImageOps.grayscale(image)
    image = transforms(image)
    return image

def get_random_images(folder):
    """
    get 10 random images form a folder containing classes and images
    :param folder: folder path (probably "test")
    :return:
    """
    img_tensor_list = []
    classes_list = []

    for i in range(10):
        random_class = None
        while random_class is None:
            folder_classes = os.listdir(folder) # list content of folder
            random_class = folder_classes[np.random.randint(0, len(folder_classes) - 1)] # pick random class folder
            if random_class == ".DS_Store": random_class = None # .DS_Store can be inside the folders and is not a class
        random_class_path = os.path.join(folder, random_class) # get path of chosen class folder
        classes_list.append(random_class_path)

        class_images = os.listdir(random_class_path) # get content of class
        random_image = class_images[np.random.randint(0, len(class_images) - 1)] # pick random image
        random_image_path = os.path.join(random_class_path, random_image) # get path

        img_tensor = img_to_tensor(random_image_path, test_mean, test_std)
        img_tensor_list.append(img_tensor)
    all_img_tensor = torch.stack(img_tensor_list)
    return all_img_tensor, classes_list

def get_predictions(model, images, encoder):
    images = images.to(device) # put image into gpu (cuda) if available
    predictions = model(images).argmax(dim=1) # get model predictions
    string_predictions = list(encoder.inverse_transform(predictions.flatten().tolist())) # decode predicted labels into string classes
    return string_predictions

def make_frequency_dict(classes):
    freq_dict = {}
    for c in classes:
        if c not in freq_dict:
            freq_dict[c] = 1
        else:
            freq_dict[c] += 1
    return freq_dict

images, random_classes = get_random_images('test')
predictions = get_predictions(model, images, encoder)
actual_frequencies = make_frequency_dict(random_classes)
pred_frequencies = make_frequency_dict(predictions)

print(f"predicted class frequencies: {pred_frequencies}")
print(f"actual class frequencies: {actual_frequencies}")


predicted class frequencies: {np.str_('beignets'): 1, np.str_('panna_cotta'): 1, np.str_('hot_dog'): 1, np.str_('lobster_bisque'): 1, np.str_('ravioli'): 1, np.str_('prime_rib'): 1, np.str_('risotto'): 1, np.str_('seaweed_salad'): 1, np.str_('eggs_benedict'): 1, np.str_('mussels'): 1}
actual class frequencies: {'test\\hot_dog': 1, 'test\\chicken_curry': 1, 'test\\pho': 1, 'test\\clam_chowder': 1, 'test\\sashimi': 1, 'test\\poutine': 1, 'test\\spaghetti_carbonara': 1, 'test\\creme_brulee': 1, 'test\\club_sandwich': 1, 'test\\beet_salad': 1}


# Bonus point
Use an LLM (API) to generate a description of the food preference of a user based on 10 images that a potential user could provide. 
Please include an example of the output of your code, especially if you used an API other than the OpenAI API.

This should work well even with differing test images by setting different random seeds for the image selector.